<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [1]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access and implementations of common calculations.

In [2]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

46.069,  46.069
513.92, K 513.92
6148000.0, Pa 6148000.0
0.167, m3/kmol 0.167
0.6452,  0.6452
351.44, K 351.44
-234950000.0, J/kmol -234950000.0
-167850000.0, J/kmol -167850000.0
38744670.0, J/kmol 38744670.0
60.096,  60.096
508.31, K 508.31
4764300.0, Pa 4764300.0
0.2201, m3/kmol 0.2201
0.6689,  0.6689
355.41, K 355.41
-272420000.0, J/kmol -272420000.0
-173390000.0, J/kmol -173390000.0
39832530.0, J/kmol 39832530.0
18.015,  18.015
647.35, K 647.35
22118230.0, Pa 22118230.0
0.063494, m3/kmol 0.063494
0.348,  0.348
373.15, K 373.15
-241820000.0, J/kmol -241820000.0
-228590000.0, J/kmol -228590000.0
40656800.0, J/kmol 40656800.0


/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [3]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]
Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.11669551 1.12519412 1.85757473] 


## Adiabatic Flash Calculation
Model mass/energy balance and VLE - all variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [4]:
def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    Ltot = c.Ftot - c.Vtot  # What's not in the vapor is in the liquid
    r.V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    r.L = c.Lx * Ltot
    r.F = c.Fz * c.Ftot
    mass_balance = (r.F, r.V + r.L) # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    r.FH = p.Hl(nL=r.F, T=c.FT)
    r.VH = p.Hv(nV=r.V, T=c.flashT)
    r.LH = p.Hl(nL=r.L, T=c.flashT)
    energy_balance = (r.FH, r.VH + r.LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    VLE = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT),
                 c.Vy*c.flashP)

    # Square deviations are weighted by their approx. magnitudes and
    # summed for minimization
    return (mass_balance, energy_balance, VLE)

## What's known (static) and unknown (variable) - automatically combined and tracked.

What's known and unknown can be easily swapped around without having to modify the model.  The DotMap structures can have nested lists of DotMaps with nested lists of arrays, and so forth.

In [5]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotMap()
c.Ftot=10 # Total Feed moles
c.Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor

## Magic happens here
The VSC class combines unknown variables and static parameters into a convenient DotMap structure.  Transformations between known/unknown DotMaps and the flat arrays required by the minimization routine is automated.

The minimization algorithm is a robust Trust-Newton Conjugate Gradient coded to take advantage of JAX automatic Jacobian vector products.  

The initial JAX model compilation is a bit slow, but subsequent repeat calculations are fast.

In [6]:
vsc=VSC(c, model)
vsc.solve(verbosity=2)

3.999069398366075
4.052146650479103
5.837466043117236
5.760239475338871
5.621487910900733
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365
5.165782826388365


KeyboardInterrupt: 

In [7]:
# State parameters
vsc.sdf

vector1   vector3                    
             1         1         2         3
FT         450                              
Ftot        10                              
flashP  101325                              
Fz              0.333333  0.333333  0.333333

In [8]:
# Adjustable Variables
vsc.vdf

vector3                      vector1
               1         2         3        1
Lx      0.330391  0.332822  0.336788         
Vtot                                  0.56995
Vy      0.384579  0.338298  0.277124         
flashT                                 352.83

In [9]:
# Intermediate results
vsc.rdf



vector3                          vector1
           1         2         3            1
V   0.219191  0.192813  0.157947             
L     3.1156   3.13852   3.17593             
F    3.33333   3.33333   3.33333             
FH                               -2.70752e+06
VH                                     -70206
LH                               -2.70346e+06

In [10]:
vsc.cdf



vector1   vector3                    
              1         1         2         3
FT          450                              
Ftot         10                              
Fz               0.333333  0.333333  0.333333
Lx               0.330391  0.332822  0.336788
Vtot    0.56995                              
Vy               0.384579  0.338298  0.277124
flashP   101325                              
flashT   352.83

In [11]:
vsc.r.FH - vsc.r.VH - vsc.r.LH

DeviceArray(66143.81166754, dtype=float64)